### Goal to test binary classifier for delays, binning for target variable and give ranges for answers

In [195]:
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
sns.set(font_scale=1.4)
import copy
import pickle

In [196]:
df = pd.read_csv("data/flights_cleaned_no_outlier_iqr_with_delays.csv")

In [197]:
df.head()

,Unnamed: 0,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,op_carrier_fl_num,origin_airport_id,origin,origin_city_name,dest_airport_id,dest,dest_city_name,crs_dep_time,dep_time,dep_delay,taxi_out,wheels_off,wheels_on,taxi_in,crs_arr_time,arr_time,arr_delay,dup,crs_elapsed_time,actual_elapsed_time,air_time,flights,distance,carrier_delay,weather_delay,nas_delay,security_delay,late_aircraft_delay,weather_type,passengers,month,combined_delay
0,0,2019-10-02,AA,AA_CODESHARE,AA,5330,OH,5330,10581,BGR,"Bangor, ME",11278,DCA,"Washington, DC",543,535.0,-8.0,11.0,546.0,726.0,5.0,749,731.0,-18.0,N,126.0,116.0,100.0,1.0,590.0,0.0,0.0,0.0,0.0,0.0,Rainy,85523.0,1788.0,0.0
1,3,2019-10-02,AA,AA_CODESHARE,AA,3073,OO,3073,10581,BGR,"Bangor, ME",13930,ORD,"Chicago, IL",1328,1324.0,-4.0,17.0,1341.0,1515.0,14.0,1530,1529.0,-1.0,N,182.0,185.0,154.0,1.0,978.0,0.0,0.0,0.0,0.0,0.0,Rainy,85523.0,1788.0,0.0
2,4,2019-10-02,AA,AA_CODESHARE,AA,4023,MQ,4023,10581,BGR,"Bangor, ME",12953,LGA,"New York, NY",1449,1449.0,0.0,13.0,1502.0,1612.0,2.0,1629,1614.0,-15.0,N,100.0,85.0,70.0,1.0,378.0,0.0,0.0,0.0,0.0,0.0,Rainy,85523.0,1788.0,0.0
3,5,2019-10-02,G4,G4,G4,2119,G4,2119,10581,BGR,"Bangor, ME",14761,SFB,"Sanford, FL",1301,1308.0,7.0,7.0,1315.0,1554.0,5.0,1623,1559.0,-24.0,N,202.0,171.0,159.0,1.0,1299.0,0.0,0.0,0.0,0.0,0.0,Rainy,85523.0,1788.0,0.0
4,7,2019-10-02,AA,AA_CODESHARE,AA,4730,PT,4730,10581,BGR,"Bangor, ME",14100,PHL,"Philadelphia, PA",1001,951.0,-10.0,9.0,1000.0,1127.0,7.0,1149,1134.0,-15.0,N,108.0,103.0,87.0,1.0,473.0,0.0,0.0,0.0,0.0,0.0,Rainy,85523.0,1788.0,0.0


In [198]:
working = [ # ran with what the test data can do
    'fl_date', # get month and bin
#     'op_unique_carrier', # most extensive name list
#     'origin', # need 'origin' to merge weather but already merged! ;)
#     'dest_airport_id', # not sure about this one
    'crs_dep_time', # bin times
#     'dep_time', # only using in TRAIN, to learn how other columns affect this
#     'crs_arr_time',
#     'arr_time', # only using in TRAIN, to learn how other columns affect this
    'weather_type', # add weight values
     #'passengers', # not sure about this one
    'arr_delay' # so we can make a target column...
]

In [199]:
df_ = df.filter(items = working)

In [200]:
df_

,fl_date,crs_dep_time,weather_type,arr_delay
0,2019-10-02,543,Rainy,-18.0
1,2019-10-02,1328,Rainy,-1.0
2,2019-10-02,1449,Rainy,-15.0
3,2019-10-02,1301,Rainy,-24.0
4,2019-10-02,1001,Rainy,-15.0
...,...,...,...,...
2073110,2019-01-01,1835,Sunny,-10.0
2073111,2019-01-01,1039,Sunny,-30.0
2073112,2019-01-01,1353,Rainy,-11.0
2073113,2019-01-01,1955,Rainy,-19.0


In [201]:
# Time weight: 0-500 = 1, 501-1000 = 8, 1001-1500 = 10, 1501-2000 = 8, 2001 > = 5
df_.crs_dep_time = df_.crs_dep_time // 100
crs_dep_time_remap = {
    0: 0.10,
    1: 0.10, 
    2: 0.10,
    3: 0.10, 
    4: 0.10,
    5: 0.10, 
    6: 1,
    7: 1, 
    8: 1,
    9: 1, 
    10: 1,
    11: 1, 
    12: 1,
    13: 1, 
    14: 1,
    15: 1, 
    16: 1,
    17: 1,
    18: 1,
    19: 1, 
    20: 1,
    21: 0.50, 
    22: 0.50, 
    23: 0.50
}
df_["dep_time_hour_weight"] = df_.crs_dep_time.map(crs_dep_time_remap)

In [202]:
df_["month"] = [ i [5:7] for i in df_.fl_date ]

In [203]:
df_ = df_.drop(labels="fl_date", axis=1)

In [204]:
df_ = df_.drop(labels="crs_dep_time", axis=1)

In [205]:
month_remap = { 
    '10': 0.5,
    '11': 0.75, 
    '12': 1,
    '01': 0.75
}
df_["month_weight"] = df_.month.map(month_remap)

In [206]:
df_ = pd.get_dummies(df_, columns=['weather_type'], drop_first=True)

In [207]:
df_ = df_.drop(labels="month", axis=1)

In [208]:
df_.fillna(0,inplace=True)

In [209]:
# df_['binned']=pd.cut(x=df_['arr_delay'], bins=[-45,-30,-15,0,15,30,45],labels = [0,1,2,3,4,5])

In [210]:
tcriteria = [df_['arr_delay'].between(-10000, 0), df_['arr_delay'].between(0, 10000)]
values = [0,30]
df_['arr_delay'] = np.select(tcriteria, values, 0)

In [211]:
df_

,arr_delay,dep_time_hour_weight,month_weight,weather_type_Rainy,weather_type_Snowy,weather_type_Sunny
0,0,0.1,0.50,1,0,0
1,0,1.0,0.50,1,0,0
2,0,1.0,0.50,1,0,0
3,0,1.0,0.50,1,0,0
4,0,1.0,0.50,1,0,0
...,...,...,...,...,...,...
2073110,0,1.0,0.75,0,0,1
2073111,0,1.0,0.75,0,0,1
2073112,0,1.0,0.75,1,0,0
2073113,0,1.0,0.75,1,0,0


In [212]:
X = df_.drop('arr_delay',axis=1) # Features
y = df_.arr_delay # Target variable

In [213]:
# X=(X-X.min())/(X.max()-X.min())
# y.replace([np.inf, -np.inf], np.nan, inplace=True)
# y.dropna()
# X.replace([np.inf, -np.inf], np.nan, inplace=True)
# X.dropna()
y

0          0
1          0
2          0
3          0
4          0
          ..
2073110    0
2073111    0
2073112    0
2073113    0
2073114    0
Name: arr_delay, Length: 2073115, dtype: int64

In [214]:
y.head(50)

0      0
1      0
2      0
3      0
4      0
5      0
6      0
7      0
8      0
9      0
10     0
11     0
12    30
13     0
14    30
15     0
16     0
17     0
18     0
19     0
20     0
21     0
22    30
23     0
24     0
25    30
26     0
27     0
28     0
29     0
30     0
31     0
32    30
33     0
34     0
35    30
36     0
37     0
38     0
39     0
40     0
41     0
42     0
43    30
44     0
45     0
46    30
47     0
48     0
49     0
Name: arr_delay, dtype: int64

In [215]:
# importing train_test_split from sklearn
from sklearn.model_selection import train_test_split
# splitting the data
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

In [216]:
# importing module
from sklearn.linear_model import LinearRegression
# creating an object of LinearRegression class
LR = LinearRegression()
# fitting the training data
LR.fit(x_train,y_train)

LinearRegression()

In [217]:
y_prediction =  LR.predict(x_test)
y_prediction

array([5.08317028, 4.92419654, 4.92419654, ..., 8.84347652, 4.67950374,
       5.08317028])

In [218]:
# importing r2_score module
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
# predicting the accuracy score
score=r2_score(y_test,y_prediction)
print( 'r2 score is',score)
print('mean_sqrd_error is ==',mean_squared_error(y_test,y_prediction))
print('root_mean_squared error of is ==',np.sqrt(mean_squared_error(y_test,y_prediction)))

r2 score is 0.006841228530647836
mean_sqrd_error is == 137.77564943879722
root_mean_squared error of is == 11.737787246274198
